This ipynb is for testing GPT-3 on politcian tweets dataset using OpenAI API. The API key is omitted on this repo for security reasons, so the code won't be able to run without it.

In [ ]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 991 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 10.5 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=8a389417d0361a290487921974adca6f169b5bbe937eceb3c0523b66893d0721
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


In [ ]:
import openai
import time
import os
import pandas as pd
from pandas import DataFrame as df
from tqdm import tqdm # for progress bar on classify()
from openai.error import RateLimitError # need to catch error for sending too many inputs to api endpoint


In [ ]:
# Use this api key and model id to get the fine-tuned Currie model on Kenny's account
# API for reference: https://beta.openai.com/docs/introduction/overview

OPENAI_API_KEY = ''
MODEL_ID = 'curie:ft-personal:kenny-2022-12-05-22-56-57'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY


In [ ]:
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

In [ ]:
# Information about model
openai.Model.retrieve(MODEL_ID)

<Model model id=curie:ft-personal:kenny-2022-12-05-22-56-57 at 0x7f464e072b80> JSON: {
  "created": 1670281017,
  "id": "curie:ft-personal:kenny-2022-12-05-22-56-57",
  "object": "model",
  "owned_by": "user-ypuc5pkpnmenuviu5z9ehvwp",
  "parent": "curie:2020-05-03",
  "permission": [
    {
      "allow_create_engine": true,
      "allow_fine_tuning": true,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1670281017,
      "group": null,
      "id": "snapperm-hniCnmUTAHh9wohEcQJJW8mH",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "org-rvWBJysoXs405lvjgSp1U88F"
    }
  ],
  "root": "curie:2020-05-03"
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#LOAD POLITICIANS TWEETS
twt_load_dir = '/content/drive/MyDrive/Twitter/'

# tweets regarding the politicans
mehmet_oz_df = pd.read_csv(twt_load_dir+"adam_laxalt.csv", encoding =DATASET_ENCODING)
john_fetterman_df = pd.read_csv(twt_load_dir+"john_fetterman.csv")
adam_laxalt_df = pd.read_csv(twt_load_dir+"adam_laxalt.csv")
catherine_cortez_masto_df = pd.read_csv(twt_load_dir+"catherine_cortez_masto.csv")
ron_johnson_df = pd.read_csv(twt_load_dir+"ron_johnson.csv")
mandela_barnes_df = pd.read_csv(twt_load_dir+"mandela_barnes.csv")
donald_bolduc_df = pd.read_csv(twt_load_dir+"donald_bolduc.csv")
maggie_hassan_df = pd.read_csv(twt_load_dir+"maggie_hassan.csv")
ted_budd_df = pd.read_csv(twt_load_dir+"ted_budd.csv")
cheri_beasly_df = pd.read_csv(twt_load_dir+"cheri_beasly.csv")
joe_pinion_df = pd.read_csv(twt_load_dir+"joe_pinion.csv")
charles_schumer_df = pd.read_csv(twt_load_dir+"charles_schumer.csv")
jd_vance_df = pd.read_csv(twt_load_dir+"jd_vance.csv")
tim_ryan_df = pd.read_csv(twt_load_dir+"tim_ryan.csv")

In [ ]:
mehmet_oz_df['Tweet'] = [t+'\n\n###\n\n' for t in mehmet_oz_df['Tweet']]
john_fetterman_df['Tweet'] = [t+'\n\n###\n\n' for t in john_fetterman_df['Tweet']]
adam_laxalt_df['Tweet'] = [t+'\n\n###\n\n' for t in adam_laxalt_df['Tweet']]
catherine_cortez_masto_df['Tweet'] = [t+'\n\n###\n\n' for t in catherine_cortez_masto_df['Tweet']]
ron_johnson_df['Tweet'] = [t+'\n\n###\n\n' for t in ron_johnson_df['Tweet']]
mandela_barnes_df['Tweet'] = [t+'\n\n###\n\n' for t in mandela_barnes_df['Tweet']]
donald_bolduc_df['Tweet'] = [t+'\n\n###\n\n' for t in donald_bolduc_df['Tweet']]
maggie_hassan_df['Tweet'] = [t+'\n\n###\n\n' for t in maggie_hassan_df['Tweet']]
ted_budd_df['Tweet'] = [t+'\n\n###\n\n' for t in ted_budd_df['Tweet']]
cheri_beasly_df['Tweet'] = [t+'\n\n###\n\n' for t in cheri_beasly_df['Tweet']]
joe_pinion_df['Tweet'] = [t+'\n\n###\n\n' for t in joe_pinion_df['Tweet']]
charles_schumer_df['Tweet'] = [t+'\n\n###\n\n' for t in charles_schumer_df['Tweet']]
jd_vance_df['Tweet'] = [t+'\n\n###\n\n' for t in jd_vance_df['Tweet']]
tim_ryan_df['Tweet'] = [t+'\n\n###\n\n' for t in tim_ryan_df['Tweet']]

In [ ]:
# Use this to get completion of given prompt and model
def completion(prompt,model):
  response = openai.Completion.create(
              model=model,
              prompt=prompt+"\n\n###\n\n",
              max_tokens=1,
              temperature=0
            )
  return response.to_dict()['choices'][0]['text'][1:]

# Use this to get classifications of all political tweets. Returns % positive
def classify(model,df):
  tqdm.pandas(desc="Classfication Progress")
  total_pos = 0
  total = 0
  tweets = df['Tweet']
  
  for tweet in tqdm(tweets,total=len(tweets)):
    total += 1
    # Can only perform 60 completions per minute, so we must handle RateLimitError to set a cool-down time
    while True:
      try:
        result = completion(tweet,model)
      except RateLimitError:
        #print('Encountered RateLimitError: Waiting for 65 seconds...')
        print(f'Current Positive Rate: {total_pos/total}')
        time.sleep(65)
        continue
      break 

    if 'positive' in result: # currie will return result as string in the form of 'positive###' or 'negative###'
      total_pos += 1

  return total_pos/len(df)

In [ ]:
# Run this to begin classification 
# This will take about (# of tweets / 60) minutes to complete
print("Oz:")
classify(MODEL_ID,mehmet_oz_df)

Oz:


100%|██████████| 220/220 [00:24<00:00,  9.09it/s]


0.6590909090909091

In [ ]:
print("Fetterman:")
classify(MODEL_ID,john_fetterman_df)

Fetterman:


100%|██████████| 500/500 [00:33<00:00, 14.99it/s]


0.68

In [ ]:
print("Laxalt:")
classify(MODEL_ID,adam_laxalt_df)

Laxalt:


100%|██████████| 220/220 [00:14<00:00, 15.04it/s]


0.6590909090909091

In [ ]:
print("Cortez:")
classify(MODEL_ID,catherine_cortez_masto_df)

Cortez:


100%|██████████| 130/130 [00:09<00:00, 14.39it/s]


0.7692307692307693

In [ ]:
print("Johnson:")
classify(MODEL_ID,ron_johnson_df)

Johnson:


100%|██████████| 500/500 [00:35<00:00, 14.15it/s]


0.98

In [ ]:
print("Barnes:")
classify(MODEL_ID,mandela_barnes_df)

Barnes:


100%|██████████| 440/440 [00:28<00:00, 15.31it/s]


0.38636363636363635

In [ ]:
print("Bolduc:")
classify(MODEL_ID,donald_bolduc_df)

Bolduc:


100%|██████████| 25/25 [00:01<00:00, 13.93it/s]


0.8

In [ ]:
print("Hassan:")
classify(MODEL_ID,maggie_hassan_df)

Hassan:


100%|██████████| 130/130 [00:08<00:00, 14.68it/s]


0.19230769230769232

In [ ]:
print("Budd:")
classify(MODEL_ID,ted_budd_df)

Budd:


100%|██████████| 500/500 [00:33<00:00, 14.85it/s]


0.78

In [ ]:
print("Beasly:")
classify(MODEL_ID,cheri_beasly_df)

Beasly:


100%|██████████| 165/165 [00:11<00:00, 14.76it/s]


0.48484848484848486

In [ ]:
print("Pinion:")
classify(MODEL_ID,joe_pinion_df)

Pinion:


100%|██████████| 50/50 [00:03<00:00, 15.12it/s]


0.6

In [ ]:
print("Schumer:")
classify(MODEL_ID,charles_schumer_df)

Schumer:


100%|██████████| 175/175 [00:11<00:00, 14.83it/s]


0.8857142857142857

In [ ]:
print("Vance:")
classify(MODEL_ID,jd_vance_df)

Vance:


100%|██████████| 500/500 [00:33<00:00, 14.96it/s]


0.53

In [ ]:
print("Ryan:")
classify(MODEL_ID,tim_ryan_df)

Ryan:


100%|██████████| 500/500 [00:32<00:00, 15.16it/s]


0.58